# Raster Range
Cascade Tuholske Feb 2021 <br>

Notebook finds the pixel-level range across gridded rasters

In [1]:
#### Dependencies
import numpy as np
import pandas as pd
import rasterio 
import xarray as xr
import geopandas as gpd
import glob
import rioxarray as rio
from scipy.stats import variation 
import rasterio.mask
import matplotlib.pyplot as plt

In [2]:
def open_rst(rst_fn):
    "function opens a raster and makes it into a xarray object"
    
    rst = xr.DataArray(rasterio.open(rst_fn).read(1), dims = ['y', 'x']) # y and x are our 2-d labels
    
    return rst

In [3]:
def rst_range(rst_fns, fn_out = None, save = False):
    """ Calc pixel-level range on stacked arrays and writes it out as a raster, returns the range, max and min arrs
    fst_fns = list of rasters 
    fn_out = file name and path to write out range raster
    save = if save == true, will write raster, default = False
    
    """
    
    arr_list = [] # empty list
    
    # open and stack arrays
    for rst_fn in rst_fns:
        
      
        rst_arr = rasterio.open(rst_fn).read(1)
        arr_list.append(rst_arr)
    
    arr3d = np.dstack([arr_list])
        
    # mask zeros
    arr3d[arr3d == -999] = 0 # Set NAN to zeros
        
    # Get range
    arr_min = np.amin(arr3d, axis = 0)
    arr_max = np.amax(arr3d, axis = 0)
    arr_range = arr_max - arr_min
    arr_out = arr_range.data
    
    # get meta data & write raster 
    out_meta = rasterio.open(rst_fns[0]).meta
    out_meta.update({'dtype' : 'float64'})
    
    if save == True: 
        with rasterio.open(fn_out, "w", **out_meta) as dest:
            dest.write(arr_out, 1)
        
    return arr_range, arr_max, arr_min

In [10]:
# write range rasters out

# File Paths & FNs --- Update by geography urban/rural for MMZ, ECU, NPL
DATA_PATH = '/Users/cascade/Github/PopGridCompare/data/'

country = '_all_MMZ'
fn_out = DATA_PATH+'interim/'+'range-MMZ.tif'
rst_fns = glob.glob('/Users/cascade/Github/PopGridCompare/data/interim/*'+country+'.tif')
rst_fns

['/Users/cascade/Github/PopGridCompare/data/interim/WP16_all_MMZ.tif',
 '/Users/cascade/Github/PopGridCompare/data/interim/GPWv4_all_MMZ.tif',
 '/Users/cascade/Github/PopGridCompare/data/interim/ESRI16_all_MMZ.tif',
 '/Users/cascade/Github/PopGridCompare/data/interim/LS15_all_MMZ.tif',
 '/Users/cascade/Github/PopGridCompare/data/interim/GHS15_all_MMZ.tif']

In [11]:
# make data
data_range, data_max, data_min = rst_range(rst_fns, fn_out = fn_out, save = True)